In [1]:
import warnings
warnings.filterwarnings('ignore')
# from datetime import datetime, date
import numpy as np
import pandas as pd
# import yfinance as yf

In [2]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
fundsSI = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

fundsSI =  fundsSI[fundsSI[' LIQUIDEZ MEDIA DIARIA'] > 250000] 
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' VPA'] > 0) & (fundsSI['P/L'] > 0)]
# fundsSI

In [5]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,RNEW4,1.32,13.418756,199.0,9.165724,2.47,3.24
2,LUPA3,1.89,8.714987,198.0,3.611104,0.87,3.88
3,MEAL3,1.82,5.982725,197.0,2.287212,0.41,3.88
4,GOAU4,10.00,32.252023,196.0,2.225202,2.66,17.38
5,HBOR3,2.64,8.386596,195.0,2.176741,0.30,10.42
6,GOAU3,10.33,32.252023,194.0,2.122171,2.66,17.38
7,HBRE3,4.82,14.570089,193.0,2.022840,0.51,18.50
8,TECN3,3.37,9.446904,192.0,1.803236,0.74,5.36
9,GGBR3,19.98,54.774447,191.0,1.741464,4.63,28.80
10,MTRE3,4.87,13.056722,190.0,1.681052,0.82,9.24


In [4]:
# linhas_com_nan = top20[top20.isna().any(axis=1)]
# linhas_com_nan

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA,CAGR LUCROS 5 ANOS
1,RNEW4,1.32,13.418756,199.0,9.165724,2.47,3.24,NaN
2,LUPA3,1.89,8.714987,198.0,3.611104,0.87,3.88,NaN
4,GOAU4,10.00,32.252023,196.0,2.225202,2.66,17.38,NaN
5,HBOR3,2.64,8.386596,195.0,2.176741,0.30,10.42,NaN
6,GOAU3,10.33,32.252023,194.0,2.122171,2.66,17.38,NaN
8,TECN3,3.37,9.446904,192.0,1.803236,0.74,5.36,NaN
9,GGBR3,19.98,54.774447,191.0,1.741464,4.63,28.80,NaN
10,MTRE3,4.87,13.056722,190.0,1.681052,0.82,9.24,NaN
11,GGBR4,21.18,54.774447,189.0,1.586140,4.63,28.80,NaN
14,POSI3,7.18,17.610473,186.0,1.452712,1.35,10.21,NaN


### Porque 22,5?

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é, na verdade, o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce.

Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos.

O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores.

Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

No próximo tópico, abordaremos a calculadora do preço justo usando a Fórmula de Graham.